<a href="https://colab.research.google.com/github/DanielWarfield1/MLWritingAndResearch/blob/main/GroundX_Court_case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GroundX-Court Case
a tutorial of creating a chat bot for reading legal transcript

## Installing Dependencies

In [1]:
!pip install groundx-python-sdk

In [15]:
!pip install openai

## Setting Credentials

In [3]:
#copying from google drive to local
from google.colab import drive
drive.mount('/content/drive')

!cp '/content/drive/My Drive/ColabDatasets/Supreme Court Dobbs Transcript of Arguments.pdf' 'Dobbs.pdf'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
"""Defining GroundX Credentials
"""

from groundx import Groundx, ApiException

with open ("/content/drive/My Drive/Colab Notebooks/Credentials/groundXAPIKey.txt", "r") as myfile:
    groundx = Groundx(
        api_key=myfile.read(),
    )
print('API Key Loaded!')

API Key Loaded!


What to put in the actual article:

```
from groundx import Groundx

groundx = Groundx(
    api_key=GROUNDX_API_KEY,
)

```

In [5]:
import os
with open ("/content/drive/My Drive/Colab Notebooks/Credentials/OpenAI-danielDemoKey.txt", "r") as myfile:
    OPENAI_API_TOKEN = myfile.read()
    os.environ["OPENAI_API_KEY"] = OPENAI_API_TOKEN
print('API Key Loaded!')

import openai
openai.api_key = OPENAI_API_TOKEN

API Key Loaded!


What to put in the article:
```
import openai

openai.api_key = OPENAI_API_KEY
```

# Cleaning up bucket
deleting stuff

In [6]:
response = groundx.documents.list()
print(response.body)

{'document': {'nextToken': 'MjY5ODQx'}, 'documents': [{'documentId': 'd7a12571-a029-4270-98ac-35746e5272e9', 'fileName': 'Dobbs ', 'fileSize': '17.6 MB', 'fileType': 'pdf', 'bucketId': 6522, 'processId': '58b3273b-3d47-4e3f-b985-060fc30f34a2', 'sourceUrl': 'https://upload.eyelevel.ai/prod/file/1e761d6f-e91b-438f-bbdb-e2d6d906955f/Dobbs+', 'status': 'complete'}]}


In [12]:
# groundx.documents.delete('c931570d-3940-4a33-9e91-f1e7df558892')

## Getting Existing Bucket
should have one automatically created (the ability to declare a new bucket programatically would be pretty cool)

In [13]:
#listing currently available buckets
bucket_response = groundx.buckets.list()
print(bucket_response.body["buckets"])

[{'created': '2023-10-19T15:41:48Z', 'bucketId': 6522, 'name': 'default bucket', 'updated': '2023-10-26T14:42:23Z'}]


## Uploading a document
https://www.supremecourt.gov/oral_arguments/argument_transcripts/2021/19-1392_bq7d.pdf

In [14]:
"""Adding local .pdf file to bucket
"""

#adding a transcript of the Dobbs supreme court case to the default bucket.
ingest = groundx.documents.upload_local(body=[{
    "blob": open("Dobbs.pdf", "rb"),
    "metadata": {
        "bucketId": 6522,
        "fileName": "Dobbs ",
        "fileType": "pdf"}
    }]
)

#getting the process id
processId = ingest.body['ingest']['processId']
print('document upload process Id:')
print(processId)

document upload process Id:
58b3273b-3d47-4e3f-b985-060fc30f34a2


In [15]:
#still processing document
print(groundx.documents.get_processing_status_by_id(process_id=processId).body['ingest']['progress'])

{'processing': {'documents': [{'documentId': 'd7a12571-a029-4270-98ac-35746e5272e9', 'fileName': 'Dobbs ', 'fileSize': '556 KB', 'fileType': 'pdf', 'bucketId': 6522, 'processId': '58b3273b-3d47-4e3f-b985-060fc30f34a2', 'sourceUrl': 'https://eyelevel-upload.s3.us-west-2.amazonaws.com/prod/file/1e761d6f-e91b-438f-bbdb-e2d6d906955f/Dobbs ', 'status': 'processing'}], 'total': 1}}


In [18]:
#done processing document
print(groundx.documents.get_processing_status_by_id(process_id=processId).body['ingest']['progress'])

{'complete': {'documents': [{'documentId': 'd7a12571-a029-4270-98ac-35746e5272e9', 'fileName': 'Dobbs ', 'fileSize': '17.6 MB', 'fileType': 'pdf', 'bucketId': 6522, 'processId': '58b3273b-3d47-4e3f-b985-060fc30f34a2', 'sourceUrl': 'https://upload.eyelevel.ai/prod/file/1e761d6f-e91b-438f-bbdb-e2d6d906955f/Dobbs+', 'status': 'complete'}], 'total': 1}}


## Retreival

In [22]:
content_response = groundx.search.content(id=6522, search={"query": "What arguments were posed in banning abortions?"})
content_response.body['search']['results']

[{'documentId': 'd7a12571-a029-4270-98ac-35746e5272e9',
  'score': 145.8995,
  'sourceUrl': 'https://eyelevel-upload.s3.us-west-2.amazonaws.com/prod/file/1e761d6f-e91b-438f-bbdb-e2d6d906955f/Dobbs ',
  'text': "that the law applies when an unborn child is undeniably human , when risks to women surge , and when the common abortion procedure is brutal .\nThe lower courts held that because the law\nprohibits abortions before viability , it is unconstitutional no matter what .\nRoe and Casey's core holding ,\naccording to those courts , is that the people can protect an unborn girl's life when she just barely can survive outside the womb but not any earlier when she needs a little more help . That is the world under Roe and Casey .\nThat is not the world the Constitution\npromises . The Constitution places its trust in the people . On hard issue after hard issue , the people make this country work . Abortion is a"},
 {'documentId': 'd7a12571-a029-4270-98ac-35746e5272e9',
  'score': 68.9535

## Augmentation

In [27]:
results = content_response.body["search"]

maxInstructCharacters = 3000

llmText = ""
for r in results["results"]:
  if "text" in r and len(r["text"]) > 0:
    if len(llmText) + len(r["text"]) > maxInstructCharacters:
      break
    elif len(llmText) > 0:
      llmText += "\n"
    llmText += r["text"]

llmText

"that the law applies when an unborn child is undeniably human , when risks to women surge , and when the common abortion procedure is brutal .\nThe lower courts held that because the law\nprohibits abortions before viability , it is unconstitutional no matter what .\nRoe and Casey's core holding ,\naccording to those courts , is that the people can protect an unborn girl's life when she just barely can survive outside the womb but not any earlier when she needs a little more help . That is the world under Roe and Casey .\nThat is not the world the Constitution\npromises . The Constitution places its trust in the people . On hard issue after hard issue , the people make this country work . Abortion is a\noverrule the stare decisis section of Casey , you say . And that that's that's what I think\nis being brought up , and maybe I haven't made it\nclearer ,\nbut I've tried to .\nMS . RIKELMAN : Yes , Your Honor . I\nthink the point that the Court was making was that the fact that some st

## Generation

In [28]:
from openai import OpenAI

instruction =  "You are a helpful virtual assistant that answers questions using the content below. Your task is to create detailed answers to the questions by combining your understanding of the world with the content provided below. Do not share links."

client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-3.5-turbo-0301",
  messages=[
    {
      "role": "system",
      "content": """%s
===
%s
===
"""
      % (instruction, llmText),
    },
    {"role": "user", "content": "What arguments were posed in banning abortions?"},
  ],
)

In [29]:
completion.choices[0].message.content

"The arguments posed in banning abortions are that the law applies when an unborn child is undeniably human, when risks to women surge, and when the common abortion procedure is considered brutal. The lower courts held that prohibiting abortions before viability is unconstitutional regardless of the circumstances. However, some argue that this overrules the principle of stare decisis in Casey, meaning that some states may continue to enact laws in the teeth of the Court's precedent and never be enough of a reason to overrule the law. The Constitution places its trust in the people, according to some, and on hard issue after hard issue, the people make this country work. Abortion is a fundamental problem here as it purposefully leads to the termination of millions of human lives and has no basis in the Constitution."